In [8]:
import math
import random
from tqdm import tqdm
import math
import numpy as np
from submodlib import FacilityLocationFunction, GraphCutFunction, DisparityMinFunction, DisparitySumFunction
import pickle
import time
import os
import timeit
from itertools import permutations
from torchvision import datasets, transforms
import pandas as pd

In [2]:
dataset = "cifar10"

train_dataset = datasets.CIFAR10(root="../../data", train=True, download=True)
test_dataset = datasets.CIFAR10(root="../../data", train=False, download=True)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# import sys
# sys.path.append('..')
# from utils.feature_extrater.cifar_feature_extractor import extract_features_threaded_worker, extract_features_resnet_threaded_cifar

In [3]:
def generate_permutations(data):
    """Generates all permutations of a list."""
    for permutation in permutations(data):
        yield permutation

In [13]:
metric = "cosine"

In [17]:
func_list = ["facility-location", "disparity-min",  "disparity-sum", "graph-cut"]
subset_fraction_size = [0.5, 0.6, 0.5]

for order in generate_permutations(func_list):
    per_func_list = list(order)
    filename = ""
    for func in per_func_list:
        filename += func+"_"
    filename = filename[:-1]

    with open(f"../../../milo-base/cifar10/dataframe.pkl", "rb") as f:
        df = pickle.load(f)
        
    groups = df.groupby('Label')
    dataframes = [group for _, group in groups]

    for i, df in enumerate(dataframes):
        df["Features"] = df["Features"].to_numpy()
        df["Index"] = df["Index"].to_numpy()


    list_indexes = []
    for idx in range(len(func_list)-1):
        fraction_size = subset_fraction_size[idx]
        final_indexes = []
        for i, df in enumerate(dataframes):
            features = df["Features"].to_numpy()
            indexes = df["Index"].to_numpy()

            print("start", features.shape, indexes.shape, 30-10*idx, idx)

            # can choose a different strategy 
            func = func_list[idx+i%2]

            if func=="facility-location":
                obj = FacilityLocationFunction(n=features.shape[0], data=features, separate_rep=False, mode="dense", metric="cosine")
            elif func=="disparity-min":
                obj = DisparityMinFunction(n=features.shape[0], data=features, mode="dense", metric="cosine")
            elif func=="disparity-sum":
                obj = DisparitySumFunction(n=features.shape[0], data=features, mode="dense", metric="cosine")
            elif func=="graph-cut":
                obj = GraphCutFunction(n=features.shape[0], data=features, mode="dense", metric="cosine", lambdaVal=0.45)
            else:
                raise Exception("Sorry, no submodlib function defined")
            
            S = obj.maximize(int(fraction_size*features.shape[0]), optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, epsilon=0.1, verbose=False, show_progress=True, costs=None, costSensitiveGreedy=False)
            # S = obj.maximize(30-10*idx, optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, epsilon=0.1, verbose=False, show_progress=True, costs=None, costSensitiveGreedy=False)
            S = list(map(lambda tuple: tuple[0], S))

            print(type(S))
            indexes = indexes[S]
            features = features[S]

            final_indexes.extend(list(indexes))

            _df = pd.DataFrame()
            _df["Features"] = features
            _df["Index"] = indexes

            dataframes[i] = _df

            print("end", features.shape[0], indexes.shape[0])
            break

        list_indexes.append(final_indexes)
    print(list_indexes)
        
    with open(f"permutation_subsets/{filename}.pkl", "wb") as f:
        pickle.dump(list_indexes, f)